In [5]:
import pandas as pd
import os
import shutil
from datetime import datetime

In [27]:
shift_dir = "./../../data/4_shift/"
rem_i_dir = "./../../data/5_remove_invalid/"

Create invalid-valid table

In [7]:
all_dates = []
for fl in os.listdir(shift_dir):
    df = pd.read_csv(shift_dir + fl, index_col=0)
    df.index = pd.to_datetime(df.index)
    all_dates.extend(df.index)

all_dates = list(set(all_dates))

date_df = pd.DataFrame(index=all_dates, columns=[name[:-4] for name in os.listdir(shift_dir)])

for date in all_dates:
    for n in os.listdir(shift_dir):
        df = pd.read_csv(shift_dir + n, index_col=0)
        df = df.dropna()
        df = df[(df >= 0).all(1)]
        df.index = pd.to_datetime(df.index)
        if date in df.index:
            date_df.loc[date, n[:-4]] = 1
        else:
            date_df.loc[date, n[:-4]] = 0

date_df = date_df.sort_index()
date_df

C:\Users\ustym\AppData\Local\Temp\ipykernel_21980\119435340.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.index = pd.to_datetime(df.index)
C:\Users\ustym\AppData\Local\Temp\ipykernel_21980\119435340.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.index = pd.to_datetime(df.index)
C:\Users\ustym\AppData\Local\Temp\ipykernel_21980\119435340.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.index = pd.to_datetime(df.index)
C:\Users\ustym\AppData\Local\Temp\ipykernel_21980\119435340.py:4: UserWarning: Could not infer format, so each element will be parsed in

,AE,C,CL,D,INF,L,LA,NI,NII,NPL,PR,TA,TE,TI
2018-01-01,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2018-02-01,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2018-03-01,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2018-04-01,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2018-05-01,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-01,1,1,1,1,1,1,1,1,1,1,1,1,1,0
2024-01-01,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2024-02-01,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2024-03-01,0,0,0,0,1,0,0,0,0,0,1,0,0,0


Delete nagative and zeros

In [29]:
negative_dates = set()

for fl in os.listdir(shift_dir):
    local_plus = []
    df = pd.read_csv(shift_dir + fl, index_col=0)
    df.index = pd.to_datetime(df.index, format = '%m/%Y').strftime('%m/%Y')
    for i in df.index:
        year_pos = True
        for v in df.loc[i]:
            if v < 0:
                year_pos = False
                break
        if year_pos:
            local_plus.append(i)
    if not negative_dates:
        negative_dates.update(local_plus)
    else:
        negative_dates = negative_dates.intersection(local_plus)
    
non_neg_dates = sorted(negative_dates, key = lambda x: datetime.strptime(x, '%m/%Y'))

In [28]:
for fl in os.listdir(shift_dir):
    local_plus = []
    df = pd.read_csv(shift_dir + fl, index_col=0)
    df_sel = df.loc[df.index.intersection(non_neg_dates)]
    df_sel.to_csv(rem_i_dir + fl)